In [1]:
import os, sys
print(sys.path)

['/home/jdelzio/data-engineering-zoomcamp/week_5', '/home/jdelzio/anaconda3/lib/python311.zip', '/home/jdelzio/anaconda3/lib/python3.11', '/home/jdelzio/anaconda3/lib/python3.11/lib-dynload', '', '/home/jdelzio/anaconda3/lib/python3.11/site-packages']


In [2]:
# (if needed) - manually add spark paths
sys.path.append('/home/jdelzio/spark/spark-3.5.1-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip')
sys.path.append('/home/jdelzio/spark/spark-3.5.1-bin-hadoop3/python')

In [3]:
# Make sure SPARK_HOME is set to the right path
print(os.environ["SPARK_HOME"])

/home/jdelzio/spark/spark-3.5.1-bin-hadoop3


In [4]:
# Load data from parquet files created in 5.3.1
import pyspark
from pyspark.sql import SparkSession
pyspark.__version__

'3.5.1'

In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 00:22:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df = spark.read.parquet('fhvhv/2021/01')

In [8]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

## How to manipulate data in spark df

In [12]:
# subset columns, filter by column value:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
    .filter(df.hvfhs_license_num == 'HV0003')

DataFrame[pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: bigint, DOLocationID: bigint]

The above is another lazy executable. Executed when dataframe is consumed:

In [13]:
# Executed upon show
df.show()

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

Two types of operations in spark:
- Transformations (lazy)
    - select
    - filter
    - joins
    - group bys
- Actions (executed immediately)
    - show, take, head
    - write

Can use SQL directly, but spark has more flexible functions:

In [18]:
from pyspark.sql import functions as F
from pyspark.sql import types

In [15]:
# built in spark functions:
df.withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date','dropoff_date','PULocationID','DOLocationID') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-01|  2021-01-01|         230|         166|
| 2021-01-01|  2021-01-01|         152|         167|
| 2021-01-01|  2021-01-01|         233|         142|
| 2021-01-01|  2021-01-01|         142|         143|
| 2021-01-01|  2021-01-01|         143|          78|
| 2021-01-01|  2021-01-01|          88|          42|
| 2021-01-01|  2021-01-01|          42|         151|
| 2021-01-01|  2021-01-01|          71|         226|
| 2021-01-01|  2021-01-01|         112|         255|
| 2021-01-01|  2021-01-01|         255|         232|
| 2021-01-01|  2021-01-01|         232|         198|
| 2021-01-01|  2021-01-01|         113|          48|
| 2021-01-01|  2021-01-01|         239|          75|
| 2021-01-01|  2021-01-01|         181|         237|
| 2021-01-01|  2021-01-01|         236|          68|
| 2021-01-01|  2021-01-01|         256|       

In [19]:
# can use more complicated custom functions to manipulate data
def crazy_func(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 ==0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'
# turn python function into Spark user defined function:
crazy_udf = F.udf(crazy_func, returnType=types.StringType())

In [21]:
df.withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_udf(df.dispatching_base_num)) \
    .select('base_id','pickup_date','dropoff_date','PULocationID','DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  a/a7a| 2021-01-01|  2021-01-01|         230|         166|
|  a/a7a| 2021-01-01|  2021-01-01|         152|         167|
|  e/acc| 2021-01-01|  2021-01-01|         233|         142|
|  e/acc| 2021-01-01|  2021-01-01|         142|         143|
|  e/acc| 2021-01-01|  2021-01-01|         143|          78|
|  e/9ce| 2021-01-01|  2021-01-01|          88|          42|
|  e/9ce| 2021-01-01|  2021-01-01|          42|         151|
|  e/acc| 2021-01-01|  2021-01-01|          71|         226|
|  e/b3b| 2021-01-01|  2021-01-01|         112|         255|
|  e/b3b| 2021-01-01|  2021-01-01|         255|         232|
|  e/b3b| 2021-01-01|  2021-01-01|         232|         198|
|  s/b13| 2021-01-01|  2021-01-01|         113|          48|
|  s/b13| 2021-01-01|  2021-01-01|         239|          75|
|  s/af0| 2021-01-01|  2

In [22]:
spark.stop()